In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/2_data.csv")

In [ ]:
df.head()

,Date,Campaign ID,Campaign Goal Code,Campaign Ad ID,Ad Digital Channel,Ad channel partner,Ad Device,Ad Group,Ad Content ID,Ad Content Type,...,$ Spend,# Impressions,# Clicks,# Video Completes,# Social Likes,# Social Shares,# Web Visit,# Collateral View,# Product View,# Form Complete
0,12/14/2022,3,Intent to Buy,921,Search,Channel Partner 1,Tablets,18.0,NaN,Text,...,0,0,0,0,0,0,0,0,0,0
1,12/14/2022,3,Intent to Buy,921,Search,Channel Partner 1,Computers,13.0,NaN,Text,...,25,72,50,0,0,0,0,0,0,0
2,1/11/2023,3,Intent to Buy,921,Search,Channel Partner 1,Computers,13.0,NaN,Text,...,0,0,0,0,0,0,0,0,0,0
3,1/11/2023,3,Intent to Buy,921,Search,Channel Partner 1,Computers,18.0,NaN,Text,...,0,0,0,0,0,0,0,0,0,0
4,1/11/2023,3,Intent to Buy,921,Search,Channel Partner 1,Computers,20.0,NaN,Text,...,10,6,5,0,0,0,1,0,0,0


In [ ]:
missing_values = df.isnull().sum()
print("Number of missing values in each column:\n", missing_values)



Number of missing values in each column:
 Date                       0
Campaign ID                0
Campaign Goal Code         0
Campaign Ad ID             0
Ad Digital Channel         0
Ad channel partner         0
Ad Device             257918
Ad Group              257918
Ad Content ID         143989
Ad Content Type           97
Audience Type              0
$ Spend                    0
# Impressions              0
# Clicks                   0
# Video Completes          0
# Social Likes             0
# Social Shares            0
# Web Visit                0
# Collateral View          0
# Product View             0
# Form Complete            0
dtype: int64


In [ ]:
df.dropna(subset=['Ad Content Type'], inplace=True)


In [ ]:
missing_values = df.isnull().sum()
print("Number of missing values in each column:\n", missing_values)



Number of missing values in each column:
 Date                       0
Campaign ID                0
Campaign Goal Code         0
Campaign Ad ID             0
Ad Digital Channel         0
Ad channel partner         0
Ad Device             257821
Ad Group              257821
Ad Content ID         143989
Ad Content Type            0
Audience Type              0
$ Spend                    0
# Impressions              0
# Clicks                   0
# Video Completes          0
# Social Likes             0
# Social Shares            0
# Web Visit                0
# Collateral View          0
# Product View             0
# Form Complete            0
dtype: int64


In [ ]:
df = df.drop(['Ad Device','Ad Group','Ad Content ID'], axis=1)

In [ ]:
missing_values = df.isnull().sum()
print("Number of missing values in each column:\n", missing_values)



Number of missing values in each column:
 Date                  0
Campaign ID           0
Campaign Goal Code    0
Campaign Ad ID        0
Ad Digital Channel    0
Ad channel partner    0
Ad Content Type       0
Audience Type         0
$ Spend               0
# Impressions         0
# Clicks              0
# Video Completes     0
# Social Likes        0
# Social Shares       0
# Web Visit           0
# Collateral View     0
# Product View        0
# Form Complete       0
dtype: int64


In [ ]:
# Calculate the pay per click for each channel

total_cost = df.groupby('Ad Digital Channel')['$ Spend'].sum()
total_clicks = df.groupby('Ad Digital Channel')['# Clicks'].sum()


ppc = total_cost / total_clicks

print(ppc)

Ad Digital Channel
Programmatic Display    0.015130
Search                  0.045700
Social                  0.011899
dtype: float64


In [ ]:

# Calculate the click through rate for each channel

total_impressions = df.groupby('Ad Digital Channel')['# Impressions'].sum()
total_clicks = df.groupby('Ad Digital Channel')['# Clicks'].sum()

ctr = total_clicks / total_impressions

print(ctr)

Ad Digital Channel
Programmatic Display    0.199746
Search                  0.267934
Social                  0.558849
dtype: float64


In [ ]:
#initial allocation from historical data

total_spent = df.groupby('Ad Digital Channel')['$ Spend'].sum()

print(total_spent)

Ad Digital Channel
Programmatic Display    3934704
Search                  2009657
Social                   754782
Name: $ Spend, dtype: int64


In [ ]:
#Channel allocation

import numpy as np
from scipy.optimize import minimize

def objective(x):
    ppc = np.array([0.015130, 0.045700, 0.011899])  # pay per click for each channel
    ctr = np.array([0.199844, 0.267934, 0.558849])  # click through rate for each channel
    return -1 * (x.dot(ppc * ctr))  # negative sign to maximize returns


budget = 1000000  # total budget
# Define the constraints
budget = 1000000
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - budget})  # budget constraint
bounds = [(budget*0.02, budget*0.5)] * 3  # bounds for each channel


# Initial budget allocation
x0 = np.array([3937607, 2009657, 754782])

# Run the optimization algorithm
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the optimal budget allocation with channel names
channel_names = ['Programmatic Display', 'Search', 'Social']
for i in range(len(channel_names)):
    print(channel_names[i] + ': $' + str(round(result.x[i], 2)))


Programmatic Display: $188178.9
Search: $500000.0
Social: $311821.1


In [ ]:

# Calculate the pay per click for each audience


total_cost = df.groupby('Audience Type')['$ Spend'].sum()
total_clicks = df.groupby('Audience Type')['# Clicks'].sum()

ppc_at = total_cost / total_clicks

print(ppc_at)

Audience Type
Audience Type 1    0.045813
Audience Type 2    0.033061
Audience Type 3    0.006642
Audience Type 4    0.027555
Audience Type 5    0.076070
dtype: float64


In [ ]:
# Calculate the click through rate for each audience

total_impressions = df.groupby('Audience Type')['# Impressions'].sum()
total_clicks = df.groupby('Audience Type')['# Clicks'].sum()


ctr_at = total_clicks / total_impressions

print(ctr_at)

Audience Type
Audience Type 1    0.272758
Audience Type 2    0.191591
Audience Type 3    0.287331
Audience Type 4    0.133860
Audience Type 5    0.115017
dtype: float64


In [ ]:
#Spent on audience type as per historical data

total_spent_at = df.groupby('Audience Type')['$ Spend'].sum()

print(total_spent_at)



Audience Type
Audience Type 1    1903352
Audience Type 2    1452119
Audience Type 3    1528882
Audience Type 4    1208669
Audience Type 5     606121
Name: $ Spend, dtype: int64


In [ ]:
#Audience allocation for Programmatic Display

import numpy as np
from scipy.optimize import minimize

def objective(x):
    ppc = np.array([0.045813,0.033012,0.006644,0.027548,0.076070])  # pay per click for each channel
    ctr = np.array([0.272758,0.191965,0.287353,0.133932,0.115017])  # click through rate for each channel
    return -1 * (x.dot(ppc * ctr))  # negative sign to maximize returns


budget = 188179  # total budget for PD
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - budget})  # budget constraint
bounds = [(budget*0.02, budget*0.5)] * 5  # bounds for each channel


# Initial budget allocation
x0 = np.array([1903352, 1453927, 1529418,1209228,606121])

# Run the optimization algorithm
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the optimal budget allocation with audience names
channel_names = ['Audience Type 1', 'Audience Type 2','Audience Type 3','Audience Type 4','Audience Type 5']
for i in range(len(channel_names)):
    print(channel_names[i] + ': $' + str(round(result.x[i], 2)))


Audience Type 1: $94089.5
Audience Type 2: $3763.58
Audience Type 3: $3763.58
Audience Type 4: $3763.58
Audience Type 5: $82798.76


In [ ]:
#Audience allocation for Search

import numpy as np
from scipy.optimize import minimize

def objective(x):
    ppc = np.array([0.045813,0.033012,0.006644,0.027548,0.076070])  # pay per click for each channel
    ctr = np.array([0.272758,0.191965,0.287353,0.133932,0.115017])  # click through rate for each channel
    return -1 * (x.dot(ppc * ctr))  # negative sign to maximize returns


budget = 500000.0  # total budget for search
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - budget})  # budget constraint
bounds = [(budget*0.02, budget*0.5)] * 5  # bounds for each channel


# Initial budget allocation
x0 = np.array([1903352, 1453927, 1529418,1209228,606121])

# Run the optimization algorithm
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the optimal budget allocation with audience names
channel_names = ['Audience Type 1', 'Audience Type 2','Audience Type 3','Audience Type 4','Audience Type 5']
for i in range(len(channel_names)):
    print(channel_names[i] + ': $' + str(round(result.x[i], 2)))


Audience Type 1: $250000.0
Audience Type 2: $10000.0
Audience Type 3: $10000.0
Audience Type 4: $10000.0
Audience Type 5: $220000.0


In [ ]:
#Audience allocation for Social

import numpy as np
from scipy.optimize import minimize

def objective(x):
    ppc = np.array([0.045813,0.033012,0.006644,0.027548,0.076070])  # pay per click for each channel
    ctr = np.array([0.272758,0.191965,0.287353,0.133932,0.115017])  # click through rate for each channel
    return -1 * (x.dot(ppc * ctr))  # negative sign to maximize returns


budget = 311821  # total budget for social
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - budget})  # budget constraint
bounds = [(budget*0.02, budget*0.5)] * 5  # bounds for each channel


# Initial budget allocation as per audience type
x0 = np.array([1903352, 1453927, 1529418,1209228,606121])

# Run the optimization algorithm
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the optimal budget allocation with audience names
channel_names = ['Audience Type 1', 'Audience Type 2','Audience Type 3','Audience Type 4','Audience Type 5']
for i in range(len(channel_names)):
    print(channel_names[i] + ': $' + str(round(result.x[i], 2)))

Audience Type 1: $155910.5
Audience Type 2: $6236.42
Audience Type 3: $6236.42
Audience Type 4: $6236.42
Audience Type 5: $137201.24
